In [1]:
import sys, os
import numpy as np
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient


def cross_entropy__error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, t.size)
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [2]:
class SoftmaxWithLoss:
    
    def __init__(self):
        self.loss = None
        self.y = None #softmaxの出力
        self.t = None #教師データ
    
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x) 
        self.loss = cross_entropy__error(self.y, self.t)
        return self.loss
    
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx


In [11]:
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Reru1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    

    def gradient(self, x, t):
        self.lastLayer = SoftmaxWithLoss()
        # forward
        network.loss(x, t)
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        #設定
        grads = {}
        grads['W1'], grads['b1'] = network.layers['Affine1'].dW, network.layers['Affine1'].db
        grads['W2'], grads['b2'] = network.layers['Affine2'].dW, network.layers['Affine2'].db
        return grads

    
class Relu:
    
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx


class Affine:
    
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
            self.original_x_shape = x.shape
            x = x.reshape(x.shape[0], -1)
            self.x = x
            out = np.dot(self.x, self.W) + self.b
            return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = dx.reshape(*self.original_x_shape)
        return dx    



In [12]:
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]


# 数値微分
grad_numerical = network.numerical_gradient(x_batch, t_batch)
# Backward

grad_backprop = network.gradient(x_batch, t_batch)
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ": " + str(diff))

W1: 2.454721176611949e-13
b1: 9.002082824831337e-13
W2: 9.05609840502091e-13
b2: 1.1923794868140546e-10


In [13]:
from dataset.mnist import load_mnist
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 1

train_loss_list = []
train_acc_list = []
test_acc_list = []

item_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #勾配
    # grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    #更新
    for key in ('W1', "b1", "W2", "b2"):
        network.params[key] -= learning_rate * grad[key] # 勾配方向にパラメータを進めて減少関数の値を減らすようにする
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % item_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train_acc, test_acc | " + str(train_acc) + ", " + str(test_acc))

train_acc, test_acc | 0.09071666666666667, 0.0897
train_acc, test_acc | 0.9458666666666666, 0.9451
train_acc, test_acc | 0.9496166666666667, 0.9439
train_acc, test_acc | 0.9600666666666666, 0.9511
train_acc, test_acc | 0.9685333333333334, 0.9618
train_acc, test_acc | 0.9695333333333334, 0.9616
train_acc, test_acc | 0.9716833333333333, 0.9611
train_acc, test_acc | 0.97595, 0.9645
train_acc, test_acc | 0.97445, 0.9609
train_acc, test_acc | 0.9745166666666667, 0.9648
train_acc, test_acc | 0.9802833333333333, 0.9658
train_acc, test_acc | 0.9791333333333333, 0.9655
train_acc, test_acc | 0.98055, 0.9662
train_acc, test_acc | 0.9792166666666666, 0.9646
train_acc, test_acc | 0.9826333333333334, 0.9655
train_acc, test_acc | 0.9825, 0.9662
train_acc, test_acc | 0.98365, 0.9639


# 感想
数値微分と誤差逆伝播法を比較した結果，誤差は10のマイナス10乗ほどになり，誤差が非常に小さいことが分かった。また，誤差逆伝播法を用いても十分な精度が出ることが分かった。計算グラフは，前後の動作が分からなくても計算ができるという点で，undo，redoと似ているように感じた。

# 参考文献
- [【ゼロから作るDeep Learning】5章 誤差逆伝播法](http://yusuke-ujitoko.hatenablog.com/entry/2016/12/28/155150)